# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.15it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.07it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.56it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.33it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.31it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  5.67it/s]

 78%|███████▊  | 18/23 [00:03<00:00,  5.64it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.65it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  5.71it/s]

100%|██████████| 23/23 [00:04<00:00,  4.76it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Sante Applegate and I am excited to be part of the Lee’s Summit R-7 School District team as a school counselor at Lee’s Summit High School. This will be my 14th year in education, and I am passionate about helping students succeed academically, personally, and professionally.
Prior to joining the district, I worked as a counselor in several schools in the Kansas City area. I have a Master’s degree in School Counseling from the University of Kansas and a Bachelor’s degree in Psychology from Benedictine College.
I am committed to providing a supportive and inclusive environment for all students. As a school counselor, my goal
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States, and is the most powerful person in the world. The president is elected by the Electoral College, and serves a four-year term. The president has many responsibilities, including setting national polic

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Rev

.

 Linda

 Campbell

...

 a

 more

 nuanced

 and

 informed

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Captain Jack Sparrow, and I have been transported back in time to the year 1777. The American colonies are still under British rule, and the seeds of revolution are beginning to be sown. I am in the midst of a heist, stealing a valuable shipment of rum, when I am suddenly confronted by a group of Patriot militia. They are armed to the teeth and look like they mean business.
As the leader of the group steps forward, I can see the fire of rebellion burning in his eyes. He introduces himself as Benjamin Franklin, and I can sense the intellectual and cunning that lies beneath his humble demeanor. I quickly

Prompt: The capital of France is
Generated text:  known for its rich history, stunning architecture, and famous landmarks. It is the epicenter of French culture, cuisine, and fashion. However, the city is also known for its less-than-favorable reputation when it comes to street crime and petty theft. Visitors to Paris are often warned to be ca

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Victoria

.

 I

 am

...

 recognize

 and

 understand

 patterns

 across

In [6]:
llm.shutdown()